### Introduction

In this notebook I will be reading the book ['Python and HDF5, Unlocking Scientific Data'](http://shop.oreilly.com/product/0636920030249.do) and try implementing the code given in the book, either as is or modified to suit my understanding or try some new thing out. The notebook will be filled with a lot of code accompanied by comments mentioning some important concept, intuition. To finish the book faster and get as much content as possible in the notebook I won't be giving the background for the HDF5 file format and other high level theoretical details which can be found in the book or other pages online but focus more on practical aspect of coding in Python and using HDF5 for storing scientific data.

The HDF5 web site can be found [here](https://support.hdfgroup.org/HDF5/)

In [198]:
import numpy as np
import h5py
import os
import timeit

Lets look at the an example of data collected from weather station. Suppose we have 10 weather stations numbered from 1 to 10 for a date 1-Jan-2017 and each of them record temperature in Fahrenheit and wind speed in mph. We assume the numbers are integers and not floating point numbers

In [63]:
date = 1
month = 'Jan'
year = 2017

station_ids = range(1, 11, 1)
np.random.seed(0)
temperatures = np.asarray([np.random.randint(60, 80) for _ in station_ids])
wind_speeds = np.asarray([np.random.randint(0, 10) for _ in station_ids])

with h5py.File('weather.hdf5', 'w') as f:
    for station_id, temperature, wind_speed in zip(station_ids, temperatures, wind_speeds):        
        temperature_key = '/' + str(station_id) + '/temperature'
        f[temperature_key] = temperature
        f[temperature_key].attrs['date'] = 1
        f[temperature_key].attrs['month'] = month
        f[temperature_key].attrs['year'] = year
        wind_speed_key = '/' + str(station_id) + '/wind_speed'
        f[wind_speed_key] = wind_speed
        f[wind_speed_key].attrs['date'] = 1
        f[wind_speed_key].attrs['month'] = month
        f[wind_speed_key].attrs['year'] = year
        


Let us now open this file and retrieve readings of the station 7 and confirm its what we wrote

In [64]:
with h5py.File('weather.hdf5') as f:
    station7_temperature = f['/7/temperature']
    station7_wind_speed = f['/7/wind_speed']
    assert station7_temperature.value == temperatures[6], 'Value not same as the one written'
    assert station7_wind_speed.value == wind_speeds[6], 'Value not same as the one written'
    temperature_node_attrs = dict([a for a in station7_temperature.attrs.items()])
    print('Temperature at station 7 is', station7_temperature.value, ',wind speed recorded is'
          ,station7_wind_speed.value, ', the date these measurements were taken is',
         '%d-%s-%d'%(temperature_node_attrs['date'], temperature_node_attrs['month'], temperature_node_attrs['year']))

Temperature at station 7 is 69 ,wind speed recorded is 7 , the date these measurements were taken is 1-Jan-2017


HDF5 file is not entirely loaded in memory but only the data required and read is loaded. In above case the weathers file may have a lot of data but only the necessary information about station 7 was read in memory when requested

Let's look at another example. where we create a dataset (we are yet to see what a dataset is).


In [65]:
with h5py.File('BigArrayFile.hdf5', 'w') as f:
    dataset = f.create_dataset('big', shape = (1024, 1024), dtype = 'float32')
    
stats = os.stat('BigArrayFile.hdf5')
print('Size of the file BigArrayFile.hdf5 is',stats.st_size, 'bytes')

Size of the file BigArrayFile.hdf5 is 1400 bytes


As we see above, we created an HDF5 file and created a data set called big in it of shape $1024 \times 1024$ of type float32. Yet, the size of the file on the disk is 1400 bytes, let us set a byte at index (2, 2) with value 2.0

In [66]:
with h5py.File('BigArrayFile.hdf5') as f:
    dataset = f['big']
    dataset[2, 2] = 2.0
    
stats = os.stat('BigArrayFile.hdf5')
print('Size of the file BigArrayFile.hdf5 is',stats.st_size, 'bytes')

Size of the file BigArrayFile.hdf5 is 4195704 bytes


 As we see above, once we accessed the byte of the data set the entire dataset was flushed to the disk. The shape times 4 bytes(for float32) per location should take $1025 \times 1024 \times 8 = 4194304$, the size we see above is pretty close to this number as there HDF5 itself takes few bytes for the meta data. Also, an interesting point to note is that the dataset can be large in size (large enough to load all in memory), but only the bytes accessed will be loaded in memory.
 
 HDF5 also supports compression of the data. Lets create a dataset of same size $1024 \times 1024$, but create the dataset using compression gzip

In [67]:
with h5py.File('BigCompressedArrayFile.hdf5', 'w') as f:
    dataset = f.create_dataset('big', shape = (1024, 1024), dtype = 'float32', compression = 'gzip')
    dataset[2, 2] = 2.0
    
stats = os.stat('BigCompressedArrayFile.hdf5')
print('Size of the file BigCompressedArrayFile.hdf5 is',stats.st_size, 'bytes')

Size of the file BigCompressedArrayFile.hdf5 is 4075 bytes


As we see above, the file containing the dataset with same shape, but compressed has a much lower size of the data stored on the disk. There is however a tradeoff between space on disk and CPU time required to compress and decompress the contents. We will talk more on this later in the notebook.

We have used ``h5py.File`` to open the file. The second parameter is the ``mode`` argument which can either be

* r  : read only for existing file, fails when the provided file is not present
* r+ : read/write for existing file, fails when the provided file is not present
* w  : write, create a new file, truncates an existing file
* w- : write, same as w except that it doesnt truncate an existing file but the operation fails
* a  : read/write, if existing file not found, new one will be created (this if not the same in case of r+, but same as using either w or r+ depending on whether the file exists of not)

---

#### Drivers

HDF5 drivers are the ones who map the bytes to be written to the low level disk bytes. Following are some important types of drivers

* SEC2: Default driver that uses posix file system read/write functions to a single file
* STDIO: Use the stdio.h for performing buffered read and write functions to single file
* CORE: Performs read/write in memory for with option an option to create backing files on file system.
* FAMILY: Partitions a large file in multiple chunks
* MPIIO: Parallel HDF5 which allows multiple processes to read/write from same HDF5 file in parallel. We will see this type of driver in more details later.

---

### Datasets

In this section we will look at datasets in HDF5

Datasets are like Numpy array on disk. These datasets have a name, shape, type and can be sliced like an in memory Numpy array except that the entire dataset need not be loaded to memory and only the accessed slices from disk are loaded in memory as needed.

Let us now create a Numpy array and create a data set out of it in an HDF5 file. Lets get some attributes of the dataset and the numpy array and compare them

In [68]:
arr = np.array([[1, 2, 3, 4, 5]])
print('Numpy array type is', arr.dtype, 'shape is', arr.shape, 'type of arr is', type(arr))

with h5py.File('DatasetTest.hdf5', 'w') as f:
    f['arr'] = arr

# Opening the file again, in read mode
with h5py.File('DatasetTest.hdf5', 'r') as f:
    a = f['arr']
    print('Dataset type is', a.dtype, 'shape is', a.shape, 'type of a is', type(a))

Numpy array type is int64 shape is (1, 5) type of arr is <class 'numpy.ndarray'>
Dataset type is int64 shape is (1, 5) type of a is <class 'h5py._hl.dataset.Dataset'>




We can see the similarity between Numpy and h5py API.

Lets now read, modify the dataset and slice the dataset

In [69]:
with h5py.File('DatasetTest.hdf5', 'a') as f:
    #1
    arr = f['arr'][:]
    print('1. Arr type is', arr.dtype, 'shape is', arr.shape, 'type of a is', type(arr))
    
    #2
    arr[:] = 10
    print('2. Arr now is', arr[:], 'Dataset is', f['arr'][:])
    
    #3
    arr = f['arr'][0, 2:4]
    print('3. Arr type is', arr.dtype, 'shape is', arr.shape, 'type is', type(arr), 'contents are', arr)
    
    #4
    f['arr'][:] = 10
    print('4. Dataset now is', f['arr'][:])

1. Arr type is int64 shape is (1, 5) type of a is <class 'numpy.ndarray'>
2. Arr now is [[10 10 10 10 10]] Dataset is [[1 2 3 4 5]]
3. Arr type is int64 shape is (2,) type is <class 'numpy.ndarray'> contents are [3 4]
4. Dataset now is [[10 10 10 10 10]]



We have 4 outputs on the previous line and following are the observations

1. We can read the entire dataset in memory using `:` on the dataset object of the HDF5 file and serialize the contents to a numpy array in memory. Be careful while performing this operation on a large dataset as the entire dataset is seldom needed top be loaded in memory or in the worst case the size may be much larger than the available memory. The `...` can be used instead of `:` to reference the entire dataset.
2. Once in memory, this numpy array is just a copy and stale copy of the dataset in the file.  There is more synchronization between the array and the dataset in the file. As we see, any changes made to the contents of the array are local to the array and not reflected if we read the contents of the file back.
3. The access here to the dataset is how we will be practically accessing the HDF5 dataset in the file. We slice a part of it as we need to read and only and only this slice is serialized to memory by the HDF5 library.
4. Here we modify the entire dataset in the file. We would seldom do this in a real application especially for large datasets but only a slice of the dataset will be updated.

---

We will look at the how we can control the datatypes of the data that is read and written from and to the underlying HDF5 file.

Suppose we have a numpy array with double precision datatype float64 but we are ok to store the data as float32 to save some extra disk space effectively making the IO fast.

Let us first write the same array of double precision to the underlying file, one using double precision and one using single precision floating point type and compare the size of the files


In [70]:
arr = np.random.rand(100, 100)

with h5py.File('DoublePrecision.hdf5', 'w') as f:
    dset = f.create_dataset('arr', data = arr, dtype = 'float64')
    
with h5py.File('SinglePrecision.hdf5', 'w') as f:
    dset = f.create_dataset('arr', data = arr, dtype = 'float32')
    
dblstat = os.stat('DoublePrecision.hdf5')
singlestat = os.stat('SinglePrecision.hdf5')

print('Size of file holding float64 array is', dblstat.st_size, 
      ', Size of file holding float32 is', singlestat.st_size)

Size of file holding float64 array is 82144 , Size of file holding float32 is 42144



As we see above, the size of the file is halved by changing the datatype. This is a good optimization provided that the change of double to single precision float doesn't impact the application using the data. Similar choices can be made when using integer datatype in choosing between 8, 16 and 32 bits depending on the expected range of the values the data can take. This is no different than choosing datatypes for a variable in programming languages

Let's say we have written the float64 numpy array as a float32 array. This means when we read the value back, the datatype of the numpy array will be float32 which might not be desirable and we would want to seamlessly read the value into a float64 numpy array. There are a couple of ways we can do that as we see below.


In [71]:
with h5py.File('SinglePrecision.hdf5', 'r') as f:
    #1
    ds = f['arr']
    arr = ds[:]
    #2
    arr1 = np.empty(shape = ds.shape, dtype = 'float64')
    ds.read_direct(arr1)
    #3
    with ds.astype('float64'):
        arr2 = ds[:]

print('dtype of arr is', arr.dtype, 'dtype of arr is', arr1.dtype, 'dtype of arr2 is', arr2.dtype)
print('Equality check of arr and arr1 gives',(arr == arr1).all())
print('Equality check of arr1 and arr2 gives',(arr2 == arr1).all())

dtype of arr is float32 dtype of arr is float64 dtype of arr2 is float64
Equality check of arr and arr1 gives True
Equality check of arr1 and arr2 gives True



As we see from the above output, by default, the array is read as a float32 and the datatype of the original array written is lost. If we want to read the object back the object as a float64 array, the first approach (given in #2 above) is to initialize an empty numpy array of the desired target type and pass it to ``read_direct`` method of the HDF5 dataset instance. Another alternate(and cleaner in my opinion) is given in #3

---

Let us see the default values of an empty dataset and see how to use a different default value

In [72]:
with h5py.File('FillValue.hdf5', 'w') as f:
    dseti = f.create_dataset('empty_ints', shape = (1, 2), dtype = 'int16')
    dsetf = f.create_dataset('empty_floats', shape = (1, 2), dtype = 'float32')
    arri = dseti[:]
    arrf = dsetf[:]
    print('1. Using Defaults: arri, ', arri, 'arrf, ', arrf)
    dseti = f.create_dataset('fillv_ints', shape = (1, 2), dtype = 'int16', fillvalue = -1)
    dsetf = f.create_dataset('fillv_floats', shape = (1, 2), dtype = 'float32', fillvalue = float('nan'))
    arri = dseti[:]
    arrf = dsetf[:]
    print('2. Using FillValues: arri, ', arri, 'arrf, ', arrf)
    

1. Using Defaults: arri,  [[0 0]] arrf,  [[ 0.  0.]]
2. Using FillValues: arri,  [[-1 -1]] arrf,  [[ nan  nan]]



As we see above, the default value for int as well as float is 0. The domain of the application might have 0 as a valid value and thus we desire to different value which clearly identifies the value at that location in the array being absent and not a valid value. In such case we can use the ``fillvalue`` parameter to use a different fill value other than 0.

Suppose our application domain requires a value to be a non negative value for int fields and a non Nan value to be used for floating point numbers, it makes sense to give a negative value as a fill value for integers and a NaN for floats as a default value.

---

### Reading and writing Data

Let us create a dataset of size $100 \times 1000 $

In [73]:
arr = np.random.randn(100, 1000)
with h5py.File('Slicing.hdf5', 'w') as f:
    f['arr1'] = arr
    f['arr2'] = arr
    dset = f['arr1']
    #1
    print(dset)
    slice = dset[10:20, 20:70]
#2
print('Slice shape', slice.shape, ', Slice type', type(slice))
    

<HDF5 dataset "arr1": shape (100, 1000), type "<f8">
Slice shape (10, 50) , Slice type <class 'numpy.ndarray'>



As we see above, we have two datasets of size $100 \times 1000$
If we want a slice of it of ``dset[10:20, 20:70]`` we get a ``numpy.ndarray`` of shape `(10, 50)` and following is what happenes when we request the slice

1. The shape of the data is figured out by h5py, in this case ``(10, 50)``
2. An empty numpy array is instantiated of this size
3. Appropriate portions of the data set are selected, this may include the necessary IO operations to take place on the underlying HDF5 file.
4. Copy the data read to the empty numpy array instantiated
5. Return the numpy array

All these operations are performed before accessing any portion of the dataset on the HDF5 file. It is thus highly recommended to vectorize the operations as far as possible. Following example demonstrates vectorzation.

Suppose we want to chop the values of the dataset with values less than 0 to 0. We have the following two ways




In [74]:
%%time
#Non Vectorized
with h5py.File('Slicing.hdf5', 'a') as f:
    dset1 = f['arr1']
    
    for ix in range(100):
        for iy in range(1000):
            val = dset1[ix, iy]
            if val < 0: dset1[ix, iy] = 0
                

CPU times: user 31.1 s, sys: 2.86 s, total: 33.9 s
Wall time: 34.7 s


In [75]:
%%time
#Vectorized
with h5py.File('Slicing.hdf5', 'a') as f:
    dset2 = f['arr2']    
    
    for ix in range(100):
        val = dset2[ix, :]
        val[val < 0] = 0
        dset2[ix,:] = val

CPU times: user 61.2 ms, sys: 6.92 ms, total: 68.1 ms
Wall time: 67.6 ms



As we see above there is a signficant performance difference in updating the matrix element by element in a HDF5 dataset vs vectoring the operations along the longer dimension. Note that there is nothing stopping us from loading the entire data set in memory for the matrix of $100 \times 1000$ case above, but for real world datasets we won't be loading the entire dataset in memory for manipulation, instead we will be reading the large chunk of data in memory and vectorise the operation on along these slices in a loop. It is interesting to see how the operation using slicing effectively scales up with such a small dataset. You can imagine the performance benefits on the real world, large dataset containing millions of elements

Finally we want to ensure that two matrices contain identical data

In [76]:
with h5py.File('Slicing.hdf5', 'r') as f:
    dset1 = f['arr1'][:]
    dset2 = f['arr2'][:]
    print('Two data sets equality check gives', (dset1 == dset2).all())
    print('Negative number check gives', (dset1 >= 0).all())

Two data sets equality check gives True
Negative number check gives True


---

You might have noticed that we use indexing of datasets in HDF5 is similar to Numpy datasets. Following snippet gives some ways to access the dataset elements. 

An important thing to remember is we **cannot** read the dataset in reverse order line a Numpy array
by giving the range as ``-1:0`` also the step value has to be a non negative number where the format of the array index is ``[start]:[end]:[step]``


In [93]:
with h5py.File('Indexing.hdf5', 'w') as f:    
    dset = f.create_dataset('data', data = np.arange(10))
    #Read 3rd element
    print('1. Third element is', dset[3])
    #Read last element
    print('2. Last element is', dset[-1])
    #Read elements 2:7
    print('3. Read elements 2 thru 7', dset[2:7])
    #Read elements 4:8 step by 2
    print('4. Read elements 4 thru 8, step by 2', dset[4:8:2])
    #Read elements 4 to the end of the array
    print('5. Read elements 4 to end of the array', dset[4:])
    #Read elements 4 to second to last element, -1 is for the last element of the array
    print('6. Read elements 4 to second to last element', dset[4:-1])   
    
    

1. Third element is 3
2. Last element is 9
3. Read elements 2 thru 7 [2 3 4 5 6]
4. Read elements 4 thru 8, step by 2 [4 6]
5. Read elements 4 to end of the array [4 5 6 7 8 9]
6. Read elements 4 to second to last element [4 5 6 7 8]


---

Lets look at multidimensional slicing and the ``...`` (ellipses) operator in HDF5. For demonstrating the concept we will be creating a 4D dataset


In [115]:
with h5py.File('MultiDimSlicing.hdf5', 'w') as f:
    arr = np.random.rand(20, 10, 12, 5)
    dset1 = f.create_dataset('4d', data = arr, dtype = 'float32')
    dset2 = f.create_dataset('1d', data = 11, shape = (1,))
    dset3 = f.create_dataset('scalar', data = 10)
    
    # 1. Read the 4D data set's 0th element of in the first and fourth dimension. 
    #All other dimensions are completely read
    
    dset1_slice = dset1[0,...,0]
    print('1. Shape of the slice is', dset1_slice.shape)
    
    print('2a. Read 0th element of 1D dataset', dset2[0])
    print('2b. Read using colon', dset2[:])
    print('2c. Read using ellipses', dset2[...])
    
    #print('2a. Read 0th element of scalar dataset', dset3[0])
    #print('2b. Read using colon', dset3[:])
    print('2c. Shape of the dataset is', dset3.shape)
    print('2d. Read using ellipses', dset3[...])
    print('2e. Read using ()', dset3[()])
    

1. Shape of the slice is (10, 12)
2a. Read 0th element of 1D dataset 11
2b. Read using colon [11]
2c. Read using ellipses [11]
2c. Shape of the dataset is ()
2d. Read using ellipses 10
2e. Read using () 10



When we slice as ``dset1[0,...,0]``, all elements of  $2^{nd}$ and $3^{rd}$ dimension are returned and thus the shape of the slice read is (10, 12).

Reading the $0^{th}$ element of the 1D (of size 1 in this case) array with shape (1,) gives a scalar value, however, reading with the ellipses ``...`` or colon ``:`` returns this 1 element as an array. On other hand, reading the scalar value (data with shape ``()``) using index or colon operator will give an error (Commented out code), the same data can however be retrieved using ellipses operator and ``()`` operator


---

We will look at Boolean indexing. This is a common practice in Numpy and we did boolean indexing on this notebook earlier when demonstrating the performance of vectorized vs non vectorized code. The code ``val[val < 0] = 0`` is an example of boolean indexing where elements from the dataset are selected if the mask(``val < 0``) yields ``True``.

Similar sort or boolean indexing can be done in case of HDF5 except that in HDF5 the index of the value in the mask is ``True`` is used to reference the elements from the dataset. As a consequence, instead of using boolean indexing on a dataset with lots of True values in mask, we often make changes to the Numpy array in memory and re set the value of the entire data set.

Thus the following code snippets are identical
```
for ix in range(100):
        val = dset2[ix, :]
        val[val < 0] = 0
        dset2[ix,:] = val
```
and 

```
for ix in range(100):
        val = dset2[ix, :]
        dset2[ix,val < 0] = 0
```
The first variant is recommended (and should be benchmarked) in case the mask ``val < 0 `` gives a lots of ``True`` values

---

Another way to index elements in a dataset is using coordinate lists by passing a list of coordinates we would like to read as follows

In [125]:
with h5py.File('Indexing.hdf5', 'r') as f:
    dset = f['data']
    print('1. Access using coordinate lists gives', dset[[0, 2, 4, 6, 8]])
    print('2. Access using boolean indexing gives', dset[np.arange(10) % 2 == 0])

1. Access using coordinate lists gives [0 2 4 6 8]
2. Access using boolean indexing gives [0 2 4 6 8]



As we see above accessing the data using coordinate index and boolean indexing gives the same data. Infact, boolean index internally is converted to a similar coordinate lists to retrieve data and thus is more efficient than boolean indexing as in above cases we have 50% ``True`` values in the mask (all even values in the between 0 and 9). There are few things to remember

1. We can slice only one dimension using lists, thus a slicing like ``dset[[1, 2, 3], [4, 5, 6]]`` is not valid.
2. All items in the list be specified in increasing order, which also means we cannot select the same index value twice.

One interesting quirk is shown below. This is especially interesting as we use -1 to access the last element in the dataset


In [126]:
with h5py.File('Indexing.hdf5', 'r') as f:
    dset = f['data']
    print('Slice is', dset[[-1, 2, 3, 4]])

Slice is [2 3 4 9]



-1 accesses the last element but still needs to be placed ahead of the index 2, thus we **cannot** pass the list as ``[2, 3, 4, -1]`` which essentially is same as ``[2, 3, 4, 9]`` in above case as 9 is the last index of the list.

The result returned however is not same as the order of the coordinate list. The result is same as when we would have passed ``[2, 3, 4, 9]``. This in my opinion is non intuitive and I personally would stay away from using negative index values (as of now while I am writing this notebook)

In [139]:
with h5py.File('Slicing.hdf5', 'r') as f:
    dset = f['arr1']
    idx = np.arange(1000)
    idx = idx < 4
    #print(dset[[1, 2], [0, 1, 2, 3]])
    #print(dset[[1, 2], idx])
    print(dset[:, [1, 2]].shape)

(100, 2)



Above code snippet demonstrates that we can slice using boolean indexing or coordinate lists only in one dimension. Uncommenting either of the commented code above will give an error

---

We will first look at an concept from Numpy which lets us create a ``slice`` for taking slices of any array.
The class ``slice`` in Numpy is instantiated whenever we access any numpy array using a start stop and step values.
For e.g ``arr[1:6:2]`` takes a slice of a numpy array ``arr``. The value ``1:6:2`` is converted to a slice object which is then used to take the slice of the array ``arr``. Let's look at it in action in the following example


In [168]:
arr = np.arange(10)
print('1. Slice using 1:6:2 gives', arr[1:6:2])
s = np.s_[1:6:2]
print('2. Slicing using slice object gives', arr[s])
print('3. Slice object is', s)

1. Slice using 1:6:2 gives [1 3 5]
2. Slicing using slice object gives [1 3 5]
3. Slice object is slice(1, 6, 2)



As we see in the above code snippet, both invocations give the same result, where by internally the value ``1:6:2`` in the former way, which also is the more intuitive and common way of indexing an array, is compiled to a slice object before getting the actual slice. A slice is created in Numpy using ``np.s_`` which creates a slice object. The slice has the signature ``slice(start, stop[, step])``

This slice object can be used multiple times to slice multiple arrays. Whenever we have an array with multiple dimensions the return value of ``np.s_`` is a tuple giving a slice object or a scalar of tuples, created one for each dimension. Following couple of examples demonstrate this concept. We instantiate slices for 2 and three dimension arrays and we see the number of elements in the tuple are same as the dimension of the array we intend to slice. Whenever a scalar is used to access the element in a dimension as in case of ``s2``, a scalar is returned in the tuple

In [172]:
s1 = np.s_[0:5, 2:10:2]
s2 = np.s_[0, 1:5, 2:8]
print('s1 is', s1, 's2 is ', s2)

s1 is (slice(0, 5, None), slice(2, 10, 2)) s2 is  (0, slice(1, 5, None), slice(2, 8, None))




Now that we have introduced what ``np.s_`` does, lets revisit ``read_direct`` we briefly introduced earlier in the notebook. the ``read_direct`` method gets close to the native C implementation. 

In [192]:
with h5py.File('Slicing.hdf5', 'r') as f:
    dset = f['arr1']
    out = np.empty(dset.shape, dtype = np.float64)
    dset.read_direct(out)
    print('1. Small slice of out', out[0:2, 0:2])

1. Small slice of out [[ 0.          0.        ]
 [ 0.40741902  0.86558573]]



The above implementation reads the entire dataset in the numpy array which seldom is done on real world datasets. Instead we want to read a portion of it. Following example demonstrates how to read the entire first two rows in an empty numpy array.

In [196]:
#1. Initialize the out array of desired shape

out1 = np.empty(shape = (2, 1000), dtype = np.float64)
#2. Instantiate the slice instance(s) for each dimension giving the target indices of out1, in our case the target 
# array is of exact size as the value we plan to read, hence [:,:]
out_slice = np.s_[:,:]

#3. The slice instances for the input dataset. Since we want to read the first two rows and all columns of the 
# dataset we instantiate it as follows. Note that the shape of the slices generated by in_slice and out_slice should
# be same (if not same the value should be broadcastable, but we will skip that possibility and keep it simple)
in_slice = np.s_[[0,1], :]

with h5py.File('Slicing.hdf5', 'r') as f:
    dset = f['arr1']
    dset.read_direct(out1, source_sel = in_slice, dest_sel = out_slice)
    
assert (out1 == out[0:2, :]).all(), 'out1 == out[0:2, :] expected'



--- 

Lets do a small test of endians in Numpy.

In [204]:
le = np.ones((1000, 1000), dtype = '<f4')
be = np.ones((1000, 1000), dtype = '>f4')
print(timeit.timeit(le.mean, number = 1000))
print(timeit.timeit(be.mean, number = 1000))

0.5113998769666068
1.2289076369488612




Natively, Intel processors use Little Endian format to store multi byte numbers. The above code is exactly identical and generates same results, excapt that one of them is more that 2 as faster than other.
The array that uses little endian datatype is faster than the one using big endian. In HDF5 file both endian formats are supported, however, when loaded they are are loaded in the same format as the ones they are stored in. If they are stored in Big endian format, when loaded in memory of a computer using a Little endian format, we have to pay the price for the performance. This is just one of the performance problems we may face if the datatypes aren't considered carefully

---

Finally, lets look at resizing datasets.

Any dataset that we create without the maxshape attribute not only has a ``maxshape`` which defaults to ``shape`` but is continuous, thus any attempt to resize it will throw an exception saying *Only chunked datasets can be resized*. 

Specifying the ``maxshape`` attribute will make the dataset chunked and will allow extending the dataset to the value of the maxshape in each of the dimensions.

If we want unlimited extension, specify the value in a given dimension as None. For e.g., suppose we just want to append rows to the (2, 2) dataset, the ``maxshape`` will be (None, 2). 

In [244]:
with h5py.File('Resize.hdf5', 'w') as f:
    twobytwo = f.create_dataset('twobytwo', shape = (2, 2))
    print('1a. Shape of twobytwo is', twobytwo.shape)
    print('1b. Maxshape if twobytwo is', twobytwo.maxshape)
    #Uncommenting the following will throw an exception    
    #twobytwo.resize(1, 1)
    twobytwochunked = f.create_dataset('twobytwochunked', shape = (2, 2), maxshape = (2, 2))
    print('2a. Shape of twobytwochunked is', twobytwochunked.shape)
    print('2b. Maxshape if twobytwochunked is', twobytwochunked.maxshape)
    print('2c. Resizing twobytwochunked to (1, 2)')
    twobytwochunked.resize((1, 2))
    print('2d. Shape of twobytwochunked is', twobytwochunked.shape)
    #Uncommenting following will give an error as maxshape is (2, 2)
    #twobytwochunked.resize((3, 2))
    nbytwochunked = f.create_dataset('nbytwochunked', shape = (2, 2), maxshape = (None, 2))
    print('3a. Shape of nbytwochunked is', nbytwochunked.shape)
    print('3b. Maxshape if nbytwochunked is', nbytwochunked.maxshape)
    print('3c. Resizing nbytwochunked to (10, 2)')
    nbytwochunked.resize((10, 2))
    print('3d. Shape of nbytwochunked is', nbytwochunked.shape)
    
    #Uncommenting the following will fail as the max shape for the second dimension is 2
    #nbytwochunked.resize((10, 3))

1a. Shape of twobytwo is (2, 2)
1b. Maxshape if twobytwo is (2, 2)
2a. Shape of twobytwochunked is (2, 2)
2b. Maxshape if twobytwochunked is (2, 2)
2c. Resizing twobytwochunked to (1, 2)
2d. Shape of twobytwochunked is (1, 2)
3a. Shape of nbytwochunked is (2, 2)
3b. Maxshape if nbytwochunked is (None, 2)
3c. Resizing nbytwochunked to (10, 2)
3d. Shape of nbytwochunked is (10, 2)




Resizing dataset in HDF5 is not same as resizing a Numpy array. Lets look at resizing Numpy array of size 2, 2

In [236]:
nparr = np.array([[1, 2], [3, 4]])
print('1. Shape is array is', nparr.shape)
print('2. Contents of nparr are', nparr.tolist())
nparr.resize((1, 4))
print('3. Shape is array is', nparr.shape)
print('4. Contents of nparr are', nparr.tolist())
nparr.resize((1, 10))
print('5. Shape is array is', nparr.shape)
print('6. Contents of nparr are', nparr.tolist())


1. Shape is array is (2, 2)
2. Contents of nparr are [[1, 2], [3, 4]]
3. Shape is array is (1, 4)
4. Contents of nparr are [[1, 2, 3, 4]]
5. Shape is array is (1, 10)
6. Contents of nparr are [[1, 2, 3, 4, 0, 0, 0, 0, 0, 0]]



As we see above, when we resize, existing elements of the Numpy array are retained, they just are reordered. If the resizing exceeds the number of elements in the array, the remaining elements are defaulted to 0.

Lets see what happens when we reshape the HDF5 dataset.

In [242]:
with h5py.File('Resize2.hdf5', 'w') as f:
    origtwobytwo = f.create_dataset('origtwobytwo', data = np.array([[1, 2], [3, 4]]), maxshape = (None, None))
    print('1. Shape of origtwobytwo is', origtwobytwo.shape)
    print('2. Data of origtwobytwo is', origtwobytwo[...].tolist())
    print('3. Resizing to shape (1, 4)')
    origtwobytwo.resize((1, 4))
    print('4. Shape of origtwobytwo is', origtwobytwo.shape)
    print('5. Data of origtwobytwo is', origtwobytwo[...].tolist())


1. Shape of origtwobytwo is (2, 2)
2. Data of origtwobytwo is [[1, 2], [3, 4]]
3. Resizing to shape (1, 4)
4. Shape of origtwobytwo is (1, 4)
5. Data of origtwobytwo is [[1, 2, 0, 0]]




After resize, unlike Numpy array where the data is shuffled, in HDF5 dataset, the data is lost. Since we kept only one row in the resized dataset, the first row was retained and everything was made 0 thus losing data. This is an important fact to keep in mind when we we reduce the size of any dimension of an HDF5 data with existing data.
The values filled in this case are 0 and can be changed to any other value using the ``fillvalue`` parameter when we create the dataset.

---

### Chunking and Compression

Storing multidimensional data on disk is flattening the data and storing it as a stream of byte array. Suppose we have a dataset of shape (100, 480, 640) which holds 100 images of size $640 \times 480$. The fast moving dimension is the last dimension (of size 640) and is the one whose values are stored continuously. Thus one image is stored as 640 pixel values stored continuously 480 times which is $640 \times 480 = 307200$ bytes (assuming one byte is used to store a pixel value and the image is a gray scale as we dont have 3 channels for each pixel). 
This is just one image, such 307200 bytes will be stored 100 times for the entire dataset.

Considering we have a spinning disk drive, there is a cost in moving the head to the desired location and read a stream of bytes. Thus storing the bytes used together continuously is most efficient. 
If we want the slice ``[0, :, :]``, its effifient as all these bytes are stored continuously on the underlying storage. However, consider we are trying to access the slice ``[0, 0:64, 0:64]``. This slice will access the first 64 bytes of first 64 rows of 640 pixels. This is as good as pulling a corner $64 \times 64$ pixel block of an image. This access pattern is not the most efficient way.

What if we already knew this access pattern, is there a way we can store this data in chunks of a predetermined size and then later retrieve them efficiently?

While creating a dataset using ``create_dataset`` we can specify the chunk size. For above case where we intend to access the image chunks in $64 \times 64$ pixels, an appropriate chunk size would be (1, 64, 64). 
Internally, the image bytes will be split in chunks of $64 \times 64$ and stored in the file. A BTree index will be maintained by HDF5 which will allow us to locate the location of the chunk stored on the disk and retrieve the continuous bytes stored for the chunk.

Since chunks are a fixed set of bytes stored on the disk, we can have filters configured which intercept the read and write calls allowing us to compress and decompress the data before writing and reading from the application, transparent to the application.

Let's write a small code snippet which shows how to create chunked datasets we mentioned above

In [247]:
with h5py.File('ChunkedSimple.hdf5', 'w') as f:
    ds = f.create_dataset('simplechunked', shape = (100, 480, 640), dtype = 'int8', chunks = (1, 64, 64))
    print('1. Shape of the dataset is', ds.shape)
    print('2. Chunk size of the dataset is', ds.chunks)
    print('3. Maxshape of the dataset is', ds.maxshape)

1. Shape of the dataset is (100, 480, 640)
2. Chunk size of the dataset is (1, 64, 64)
3. Maxshape of the dataset is (100, 480, 640)



Chunks are enabled whenever we make the dataset resizable or compression is enabled. In this case, the chunk size if automatically chosen by the library. Auto chunking is useful only when the sole objective is to make the dataset resizable and compression enabled and not to do with the performance of slicing the dataset.
If slicing performance and due to disk access pattern and the way data is stored is important, specifying the necessary chunking based on the data access pattern is important.

When manually picking the chunk size, considering the following is important.

1. Larger chunks result is less total number of chunks in the dataset thus reducing the size of the chunk Btree. Smaller the tree, faster would be the lookup.
2. Chunk loading is either complete or nothing. So even if we wish to read a single location from the dataset, entire chunk will be loaded.
3. Chunk cache will not cache chunks larger than 1 MB. Even for chunks smaller than 1 MB, the number of chunks in the cache size if limited.

---

#### Filters

HDF5 has a concept of Filter Pipeline where the data written or read is passed through a sequence of filters which can perform any operation like compression, add metadata, calculate checksum etc. While reading the filters are invoked with the data in the reverse order to that of the order during write and the operation performed will be reverse of what was done, that is for compression/decompression filter, if the data was compressed during write, during read it will perfom decompression. Also, once a data set of created with a list of filters, they cannot be modified.

---

Let's first look at compression filters. The most common type of filter in use is GZIP Filter, also called ***DEFLATE*** filter. Setting up a dataset for compression is as simple as follows. Note that the compression enables chunking too.




In [252]:
with h5py.File('GZIPCompressed.hdf5', 'w') as f:
    ds = f.create_dataset('compressed', shape = (100, 100), dtype = 'float32', compression = 'gzip')
    print('1. Compression is ', ds.compression)
    print('2. Chunk size is', ds.chunks)

1. Compression is  gzip
2. Chunk size is (50, 50)



Above code snippet shows how we can simply compress the dataset. The way we read/write to dataset is independent of whether compression filter is enabled or not.

GZIP/DEFLATER filter has following features.

1. Its shipped with HDF5 Library
2. Supports all HDF5 datatypes
3. It gives moderate to slow speed compression

Another possible choice is LZF Compression which has the following advantages

1. Works with all HDF5 types
2. Fast compression and decompression

Some things to keep in mind before using LZF compression is 

1. It works only on Python Platforms as it ships with h5py.
2. Compression ratio is lower than GZIP, and thus the files would be bigger in size than GZIP compressed files.


There are a lot of [Third Party Filters](https://support.hdfgroup.org/services/contributions.html#filters) available and with different compression/decompression filters available. If you intend to distribute the HDF5 files to other consumers, you need to ensure that the clients are configured and are able to use those filters while reading/writing to the distributed files.


---

Another type of filter thats used in conjunction is ***SHUFFLE*** filter. Suppose we have a 32 bit integer datatype and the numbers stored in them are affecting mostly the lower two bytes, then the first two bytes are mostly all zeros. Compression algorithms give better compression when we have identical bytes following each other. Shuffle filter thus packs all bytes in a way such that all first bytes of the 4 byte number are together followed by the second, third and fourth. This way, the entropy is not scattered all over the place but concentrated around where the least significant bytes are packed allowing better compression.
Following are the points to remember when using SHUFFLE filter

1. Its relatively inexpensive to use this filter
2. Meaningful only if compression is enabled
3. Available in all HDF5 distributions.

Enabling shuffle filter is as east as follows

In [256]:
with h5py.File('CompressAndShuffle.hdf5', 'w') as f:
    ds = f.create_dataset('compress_and_shuffle', shape = (100, 100), 
                          dtype = 'int32', compression = 'gzip', shuffle = True)
    print('1. Compression is', ds.compression)
    print('2. Chunking size is', ds.chunks)
    print('3. Shuffle enabled is', ds.shuffle)

1. Compression is gzip
2. Chunking size is (50, 50)
3. Shuffle enabled is True



We will look at another filter which computes the checksums to validate data integrity whenever we read a dataset.
In case of corruption of HDF5 data file, we may not at times read the bytes those were written to the chunk. In such cases its desired to store the checksum of the chunk in the chunk's metadata. Whenever we read a chunk back, the checksum is computed and compared with the checksum stored in the metadata when it was written. 
HDF5 ships with a default filter for this purpose called the ***FLETCHER32*** filter as it uses 32 bit [Fletcher's Checksum](https://en.wikipedia.org/wiki/Fletcher%27s_checksum). 

Enabling checksum is as simple as the following code snippet and it has the folliwng features

1. Inbuilt and comes shipped with HDF5 stack
2. Very fast 
3. Compatible with other lossless filters configured.
4. When using ``fletcher32`` checksum filter compression is automatically enabled and auto chunking chooses the chunk sizes for the dataset unless we specify one.

In [265]:
with h5py.File('Checksum.hdf5', 'w') as f:
    ds = f.create_dataset('checksum', shape = (100, 100), dtype = 'int8', fletcher32 = True)
    print('1. Chunk size is', ds.chunks)
    print('2. Fletcher32 enabled?', ds.fletcher32)

1. Chunk size is (100, 100)
2. Fletcher32 enabled?, True


---

### Groups, Links and Iteration

So far all the datasets we create are created in the HDF5 file directly. Which is same as putting all files on the root partition of the file system. For managing our files we create the folders(and subfolders) in the root directory and place our files in it. HDF5 files have something similar where we can create a *hierarchy* of groups in the file. When ever we create a file as ``h5py.File('...', 'w')`` we are creating a group, the root. We are then free to create multiple nested groups in this file. Lets look at how we can create these groups in an HDF5 file.


In [268]:
with h5py.File('Groups.hdf5', 'w') as f:
    g1 = f.create_group('group1')
    g2 = g1.create_group('group2')
    #Alternativly and more readable, we create the group as follows and all intermediate
    #missing groups are automatically created. Kind of like mkdir -p ... in Unix
    g3 = f.create_group('/group1/group3')
    print('1. g1 is', g1)
    print('2. g2 is', g2)
    print('3. g3 is', g3)    

1. g1 is <HDF5 group "/group1" (2 members)>
2. g2 is <HDF5 group "/group1/group2" (0 members)>
3. g3 is <HDF5 group "/group1/group3" (0 members)>



With the groups created as above, lets create some datasets in thse groups created